# 02: Advanced Functions

- Functions as variables
- Higher order Functions
- Closure
- Decorators

## Functions as variables

Most likely, you have only seen functions as something you declare and call, separate from variables you can modify, set, or operate with. However, in languages like Python, a function can be treated just like any other variable (as an object).

In [32]:
# declare an empty class, just to see what a
# "normal" object will be like
class Example(object):
    pass

# print the object and its type
print(Example(), type(Example()))

# declare some functions 
def multiply_by_2(x):
    return x*2

def divide_by_2(x):
    return x/2

print(multiply_by_2, type(multiply_by_2))
print(divide_by_2, type(divide_by_2))

<__main__.Example object at 0x00000244325FEBB0> <class '__main__.Example'>
<function multiply_by_2 at 0x00000244327B8040> <class 'function'>
<function divide_by_2 at 0x00000244327B80D0> <class 'function'>


Since a function is just a variable, it can be assigned to just like any other variable. Then, you can call it just like it was a normal function.

In [33]:
fun = multiply_by_2

print(fun(1))

fun = divide_by_2

print(fun(4))

2
2.0


Variables can be passed into and from functions. If functions are just like any other variable, then they can be passed into and from functions. A function which takes or returns functions is called a higher order function.

In [34]:
def apply_operation(operation, x):
    # take a function as a parameter as if it was any
    # other parameter, and also call it just like the
    # above example
    return operation(x)

print(apply_operation(multiply_by_2, 5))
print(apply_operation(divide_by_2, 8))

def get_some_function():
    # since we can treat a function as any other variable,
    # we can return it like any other value, just like
    # saying "return 5"
    return multiply_by_2

print(apply_operation(get_some_function(), 10))


10
4.0
20


This works with functions that you don't declare too. Any function.

In [35]:
import math

print(apply_operation(math.sqrt, 16))
print(apply_operation(math.log2, 8))

4.0
3.0


In [36]:
class ExampleTwo(object):
    def __init__(self, n):
        self.n = n;

    def print_hi(self):
        for _ in range(self.n):
            print("hi")

ex = ExampleTwo(5)
fun = ex.print_hi

print("running fun:")
fun()

running fun:
hi
hi
hi
hi
hi


It works with methods too, and you can see that the instance variable `n` was still accessible.

## Closure

Hopefully the idea of functions as any other object variable has stuck. Now we'll see what kind of data is stored in each function object. Take a careful look at this example.

In [37]:
def get_distance_fn(x1, y1):
    point_1 = (x1, y1)

    def inner(x2, y2):
        return math.dist(point_1, (x2, y2))

    return inner

distance_func = get_distance_fn(5, 4)

print(distance_func(7, 4))
print(distance_func(5, -4))

2.0
8.0


We saw how `self` was preserved and we could use `self.n` in a previous example, however this appears pretty different. See what happens when calling `inner`

In [38]:
try:
    inner(5, 5)
except NameError as e:
    print("error:", e)

error: name 'inner' is not defined


This should make some sort of sense. The declaration of `inner` was indented, which means it is part of the `get_distance_fn` function. If we were to use any other variable you should be able to know that it is clearly not accessible:

In [39]:
def dummy_fn():
    not_accessible_outside = 5

try:
    print(not_accessible_outside)
except NameError as e:
    print("error:", e)

error: name 'not_accessible_outside' is not defined


Remember how variable scope works:

- Variables declared in an inner block are not accessible in outside blocks
- Variables declared in outside blocks are accessible from an inner block

The reason why `not_accessible_outside` is not accessible is because it is a local variable declared within the scope of `dummy_fn` and so it is not accessible from the global scope. Likewise, the `inner` function was declared within the scope of `get_distance_fn` and, since functions are just variables, is just a local variable to `get_distance_fn`.

However, note the second point. It's the reason why this is possible:

In [40]:
global_variable = 5

def dummy_fn_2():
    print(global_variable*2)

dummy_fn_2()

10


Since `dummy_fn_2` is declared in the global scope with `global_variable`, the scope of `dummy_fn_2` is inside the global scope, and so `global_variable` is accessible from inside `dummy_fn_2`. However, back to the first example:

```python
def get_distance_fn(x1, y1):
    point_1 = (x1, y1)

    def inner(x2, y2):
        return math.dist(point_1, (x2, y2))

    return inner
```

If you imagine the inside of `get_distance_fn` was the global scope, and `x1` and `y1` were global variables, `inner` is just a function declared within that scope. Just like how `global_variable` was accessible within `dummy_fn_2`, `x1` and `y1` are accessible from within `inner`. So, it makes perfect sense according to our scope rules that we can access `x1`, `y1`, and `point_1`.

The only strange part is that we are returning `inner`, and it is still able to use `point_1` in later function calls, even though the original `point_1` is no longer accessible after we returned from `get_distance_fn`.

However, remember that functions are just objects. It turns out that the `inner` function stores in itself the variables that were accessible to it when it was declared. Binding this environment of local variables to the specific function that was created forms a closure.

In [61]:
print(distance_func.__closure__)
print(list(x.cell_contents for x in distance_func.__closure__))

(<cell at 0x000002443286FBE0: tuple object at 0x0000024432839D00>,)
[(5, 4)]


Where are the int values? Well, since `inner` doesn't use them, why would Python bother storing them? Here are a few more examples.

In [63]:
def create_quadratic_fn(a, b, c):
    def inner(x):
        return a*x*x + b*x + c
    # this will affect the c in inner, even
    # though the assignment happened after
    c = 100
    return inner

fun = create_quadratic_fn(1, 2, 3)
print(fun(1))
print(fun.__closure__)
print(list(x.cell_contents for x in fun.__closure__))

103
(<cell at 0x000002443288E8B0: int object at 0x00007FFD0F3016A0>, <cell at 0x0000024432CB3520: int object at 0x00007FFD0F3016C0>, <cell at 0x0000024432CB3F40: int object at 0x00007FFD0F302300>)
[1, 2, 100]


In [64]:
import random

def function_repeater(fun, n):
    def inner():
        for _ in range(n):
            print(fun())
    return inner

def function_to_repeat():
    return random.randint(100, 200)

repeated = function_repeater(function_to_repeat, 5)
repeated()
print(repeated.__closure__)
print(list(x.cell_contents for x in repeated.__closure__))

186
127
121
142
172
(<cell at 0x000002443244A250: function object at 0x00000244329A3F70>, <cell at 0x000002443244AF10: int object at 0x00007FFD0F301720>)
[<function function_to_repeat at 0x00000244329A3F70>, 5]


This example combines both closure and functions as parameters. To summarize:
- Functions can be declared in places other than the global scope
- Functions can store and use the variables accessible to them in the declaring scope, even if those variables later become inaccessible otherwise

## Anonymous Functions

Notice that `function_to_repeat` is declared in the global scope, just to be used as a parameter. If we don't ever want to use `function_to_repeat`, we can use an anonymous function to give a function to `function_repeater` without adding a function to the global scope:

In [44]:
repeated = function_repeater(lambda: random.randint(100, 200), 5)
repeated()

191
101
185
137
173


The syntax for an anonymous function in Python is to write `lambda`, a parameter list, then a colon, and a single expression which is the return value. Another example:

`lambda x, y: x*y`

The restriction that you can only use a single expression is just a Python thing. In the future, we'll see JavaScript anonymous functions that are basically just any other function.

A very common usage of anonymous functions is when using functions like `map` in Python.

In [45]:
strings = [
    "    hELlo thEre ",
    "hoWs it Going ",
    " ok BYE now",
]

print("".join(
    map(lambda x: x.lower().strip()+",\n", strings)
))

hello there,
hows it going,
ok bye now,



The map function takes an iterable (such as a list) and returns a new iterable with the function given applied to each element.

`map(fn, [x1, x2, x3...]) -> [fn(x1), fn(x2), fn(x3)...]`

In this case, we're taking a list of strings and making each string lowercase, removing leading and trailing whitespace, and adding `",\n"` to the end of each string.

Since this function that performs this specific string operation is not very useful to us other than for this `map` call, we can use an anonymous function to avoid declaring a new function.

## `*args` and `**kwargs`

If you want detailed information about packing and unpacking, you can [read this too.](https://stackabuse.com/unpacking-in-python-beyond-parallel-assignment/) All you really need to know for this is what `*` and `**` mean in a function definition.

In [46]:
def f(x, *args):
    print("x:", x)
    print("args:", args)
    n = 0
    for arg in args:
        n += x*arg
    return n

# 2*1 + 2*2 + 2*3 = 2 + 4 + 6 = 12
print(f(2, 1, 2, 3))

x: 2
args: (1, 2, 3)
12


Using `*` allows you to add an arbitrary amount of *positional* arguments. First, all specified positional arguments are captured, then the rest are thrown into a tuple parameter (the name does not have to be `args`, it could be something like `nums`).

In [47]:
def g(x, y=0, **kwargs):
    print("x:", x, "y:", y, "kwargs:", kwargs)

g(1)
g(4, y=5, z=10)
g(5, a="hi", b="bye")

x: 1 y: 0 kwargs: {}
x: 4 y: 5 kwargs: {'z': 10}
x: 5 y: 0 kwargs: {'a': 'hi', 'b': 'bye'}


Using `**` allows you to add an arbitrary amount of *keyword* arguments, putting any keys that aren't already specified into a dict parameter (the name does not have to be `kwargs`).

Aside from what is about to be shown, I don't think I've ever used `*` or `**` to capture arguments though.

`*` and `**` can also be used when calling functions, in addition to declaring them.

## Decorators

Suppose you use this code to debug a function.

In [48]:
def debug_me(foo, bar, x, y):
    # do some work
    result = foo+bar+str(x*y)

    return result

def debug_me_2(foo, bar, x, y):
    print(f"foo: {repr(foo)}, bar: {repr(bar)}, x: {repr(x)}, y: {repr(y)}")
    result = foo+bar+str(x*y)
    print("return:", repr(result))
    return result

print(debug_me_2("hello", "bob", 4, 6))

foo: 'hello', bar: 'bob', x: 4, y: 6
return: 'hellobob24'
hellobob24


You think that these print statements are a bit cumbersome to type, and having just learned about higher order functions and `*args`, you write this.

In [49]:
def debugger(fun, *args, **kwargs):
    print(f"calling '{fun.__name__}' with args {args} and kwargs {kwargs}")
    result = fun(*args, **kwargs)
    print(f"'{fun.__name__}' returned with {repr(result)}")
    return result

# use the original debug_me function with no changes
print(debugger(debug_me, "hello", "bob", 4, 6))

calling 'debug_me' with args ('hello', 'bob', 4, 6) and kwargs {}
'debug_me' returned with 'hellobob24'
hellobob24


Unfortuantely this solution just transforms the problem into a different (and probably a worse) problem. Instead of adding a long print statement at the start and end of the original function, you now have to change every single time you call `debug_me` to use the `debugger` syntax, and then you'd have to change every call back after you were done, instead of just deleting the print statements like in `debug_me_2`.

However, this `debugger` function works on any function now. Since it uses `*args, **kwargs` at the end, it captures all possible arguments that you could want to give to `fun`, then unpacks `args` and `kwargs` when the actual call to `fun` is made. You could have a function that is declared to take 1 positional argument, and it would work. Or, you could have a function with 10 positional arguments and 5 keyword arguments, and it would work. Just imagine this as a big pipe of arguments: you take in `*args, **kwargs` in the definition and call the function with `*args, **kwargs`. As far as both sides know, they're just passing and receiving arguments as usual, but you're free to inspect the arguments in between.

Fortunately, there is another solution that involves another fact that arises because of the fact that functions are variables, and there are no constant variables in Python.

In [50]:
def not_a_cool_function(x):
    return x

def cooler_function(x):
    return x*x

print(not_a_cool_function(2))

not_a_cool_function = cooler_function
print(not_a_cool_function(2))

cooler_function = lambda x: x*x*x
print(cooler_function(2))

2
4
8


Of course this isn't something you should use often (especially after the rest of this lesson), but we can apply this reassignment of functions to our debugger by writing a higher order function which returns a new function that wraps a given function in print statements.

In [51]:
def get_debugger(fun):
    # fun is stored in the closure of debugged.
    # remember that the inner code is not run until it is
    # actually called, since it is just a function declaration
    def debugged(*args, **kwargs):
        print(f"calling '{fun.__name__}' with args {args} and kwargs {kwargs}")
        result = fun(*args, **kwargs)
        print(f"'{fun.__name__}' returned with {repr(result)}")
        return result

    return debugged

print("without debugger:")
print(debug_me("hello", "bob", 4, 6))
print(debug_me("hi", "joe", 1, 7))

print("with debugger:")
debug_me = get_debugger(debug_me)
print(debug_me("hello", "bob", 4, 6))
print(debug_me("hi", "joe", 1, 7))

without debugger:
hellobob24
hijoe7
with debugger:
calling 'debug_me' with args ('hello', 'bob', 4, 6) and kwargs {}
'debug_me' returned with 'hellobob24'
hellobob24
calling 'debug_me' with args ('hi', 'joe', 1, 7) and kwargs {}
'debug_me' returned with 'hijoe7'
hijoe7


Now we can easily debug any function.

In [52]:
def absurd_amount_of_arguments(a, b, c, d, e, f, g, h=1, i=2, j=5):
    return a+b+c+d+e+f+g+h+i+j

print("without debugger:")
print(absurd_amount_of_arguments(1, 2, 3, 4, 5, 6, 7, 8))
print(absurd_amount_of_arguments(1, 2, 3, 4, 5, 6, 7, 8, 9, j=1))

print("with debugger:")
absurd_amount_of_arguments = get_debugger(absurd_amount_of_arguments)
print(absurd_amount_of_arguments(1, 2, 3, 4, 5, 6, 7, 8))
print(absurd_amount_of_arguments(1, 2, 3, 4, 5, 6, 7, 8, 9, j=1))


without debugger:
43
46
with debugger:
calling 'absurd_amount_of_arguments' with args (1, 2, 3, 4, 5, 6, 7, 8) and kwargs {}
'absurd_amount_of_arguments' returned with 43
43
calling 'absurd_amount_of_arguments' with args (1, 2, 3, 4, 5, 6, 7, 8, 9) and kwargs {'j': 1}
'absurd_amount_of_arguments' returned with 46
46


Now, watch this:

In [53]:
@get_debugger
def less_arguments_this_time(a, b, c, h=False):
    if h:
        return a+b+c
    return a+b

print(less_arguments_this_time(1, 2, 3))
print(less_arguments_this_time(1, 2, 3, True))
print(less_arguments_this_time(2, 4, 1, h=True))

calling 'less_arguments_this_time' with args (1, 2, 3) and kwargs {}
'less_arguments_this_time' returned with 3
3
calling 'less_arguments_this_time' with args (1, 2, 3, True) and kwargs {}
'less_arguments_this_time' returned with 6
6
calling 'less_arguments_this_time' with args (2, 4, 1) and kwargs {'h': True}
'less_arguments_this_time' returned with 7
7


It turns out, the action of going `debug_me = get_debugger(debug_me)` is useful enough that Python has special syntax to do exactly that: `@`.

By placing `@something` in the line before a function definition `def f(x):`, you are essentially replacing `f` with `something(f)`. In this case, writing `@get_debugger` was equivalent to writing `less_arguments_this_time = get_debugger(less_arguments_this_time)`.

![Diagram of the process for decorating a function](img/simple-decorator.png)

But, this can get more complicated. What if we want to provide arguments to the decorating function? Perhaps it wouldn't be very useful for this example, but for other decorators it could be helpful. The content after `@` must be a function that takes just one argument, the function to be wrapped. So, all we need to do is *call a function which returns a decorating function*.

In [56]:
def add_some_random_results(fun):
    def wrapper(*args, **kwargs):
        result = [fun(*args, **kwargs)]
        for _ in range(4):
            result.append(random.randint(0, 100))
        random.shuffle(result)
        return result
    return wrapper

# weve_seen_this_before = add_some_random_results(weve_seen_this_before)
@add_some_random_results
def weve_seen_this_before(x, y):
    return x*y

print(weve_seen_this_before(5, 8))
print(weve_seen_this_before(2, 20))

def add_random_results(count, lower, upper):
    def decorator(fun):
        def wrapper(*args, **kwargs):
            result = [fun(*args, **kwargs)]
            for _ in range(count):
                result.append(random.randint(lower, upper))
            random.shuffle(result)
            return result
        return wrapper
    return decorator

@add_random_results(4, 0, 100)
def thats_alot_of_functions(x, y):
    return x*y

print(thats_alot_of_functions(5, 8))
print(thats_alot_of_functions(2, 20))

[64, 40, 60, 18, 64]
[95, 53, 37, 40, 51]
[10, 87, 40, 62, 17]
[3, 78, 40, 81, 49]


If you have a solid grasp of creating and returning functions, this should make sense. Something to note: the inside of the `decorator` function is not evaluated when creating the decorator with `@add_random_results(4, 0, 100)`, because the argument for `fun` is not known. All that happens is the function `decorator` is created and bound to a closure containing the arguments `count`, `lower`, and `upper`, which is then used to decorate `thats_alot_of_functions` like a normal decorator function.

![Diagram overview of decorating with arguments](img/decorator-with-args-setup.png)

![Diagram of the process for decorating a function with decorator arguments](img/decorator-with-args-following.png)

When are decorators evaluated? The decorator is created once and the wrapper is created once. Then of course the wrapper function is evaluated each time the function is called.

In [58]:
def print_something(message):
    print(f"creating decorator with {repr(message)}...")
    def decorator(fun):
        print(f"wrapping function {repr(fun.__name__)}...")
        def wrapped(*args, **kwargs):
            print(f"running wrapper...")
            print(message)
            return fun(*args, **kwargs)
        return wrapped
    return decorator

@print_something("wow!")
def print_hi(n):
    print("running original...")
    for _ in range(n):
        print("hi!")

print_hi(3)
print_hi(2)

creating decorator with 'wow!'...
wrapping function 'print_hi'...
running wrapper...
wow!
running original...
hi!
hi!
hi!
running wrapper...
wow!
running original...
hi!
hi!


In [65]:
def decorate_a(message):
    print(f"creating 'decorator_a' with {repr(message)}...")
    def decorator_a(fun):
        print(f"wrapping function {repr(fun.__name__)}...")
        def wrapper_a(*args, **kwargs):
            print(f"running 'wrapper_a'...")
            print(message)
            return fun(*args, **kwargs)
        return wrapper_a
    return decorator_a
def decorate_b(message):
    print(f"creating 'decorator_b' with {repr(message)}...")
    def decorator_b(fun):
        print(f"wrapping function {repr(fun.__name__)}...")
        def wrapper_b(*args, **kwargs):
            print(f"running 'wrapper_b'...")
            print(message)
            return fun(*args, **kwargs)
        return wrapper_b
    return decorator_b
def decorate_c(message):
    print(f"creating 'decorator_c' with {repr(message)}...")
    def decorator_c(fun):
        print(f"wrapping function {repr(fun.__name__)}...")
        def wrapper_c(*args, **kwargs):
            print(f"running 'wrapper_c'...")
            print(message)
            return fun(*args, **kwargs)
        return wrapper_c
    return decorator_c

@decorate_b("wow from b!")
@decorate_a("wow from a!")
@decorate_c("wow from c!")
def print_bye(n):
    print("running original...")
    for _ in range(n):
        print("bye!")

print_bye(3)
print_bye(2)

creating 'decorator_b' with 'wow from b!'...
creating 'decorator_a' with 'wow from a!'...
creating 'decorator_c' with 'wow from c!'...
wrapping function 'print_bye'...
wrapping function 'wrapper_c'...
wrapping function 'wrapper_a'...
running 'wrapper_b'...
wow from b!
running 'wrapper_a'...
wow from a!
running 'wrapper_c'...
wow from c!
running original...
bye!
bye!
bye!
running 'wrapper_b'...
wow from b!
running 'wrapper_a'...
wow from a!
running 'wrapper_c'...
wow from c!
running original...
bye!
bye!


## Exercises

1. Type conversion decorator

Create a decorator `def convert_types(*args)`, which can be used to decorate function like so:

```python
@convert_types(int, int, float, bool)
def fun(x, y, z, flag):
    if flag:
        return pow(x+y, int(z), 1000)
    return (x+y)**z
```

The decoration should use the provided function e.g. `int`, `float`, to convert each argument, which would allow a call such as `fun("1", 5.8, "5.9", 1)` to succeed. Optionally add support for `**kwargs`

```python
@convert_types(int, int, float, flag=bool)
def fun(x, y, z, *, flag=False):
    if flag:
        return pow(x+y, z, 1000)
    else:
        return (x+y)**z
```

2. Random function return buffer

Create a decorator `def randomly_buffer(return_chance)`, which can be used to decorate a function like so:

```python
@randomly_buffer(0.6)
def fun(x, y):
    return x*y
```

The decoration should maintain a list of values. Each time `fun` is called, the return value is added to the list. There will then be a `return_chance`% chance that `fun` returns with a list of all the stored values, otherwise `None` is returned. For example:

```python
print(fun(1, 5))  # None
print(fun(5, 4))  # None
print(fun(2, 3))  # None
print(fun(9, 4))  # [5, 20, 6, 36]
print(fun(4, 8))  # None
print(fun(2, 9))  # [32, 18]
```

3. Function cache

Create a decorator `cache` which caches function arguments.

```python
@cache
def fun(x, y):
    time.sleep(x)
    return x*y
```

The decorator only needs to work for functions which only take positional arguments. Optionally make it work for functions with keyword arguments too. If the function is called, the args that were used and the return value that was produced is stored in the decorator. When the function is called with arguments that have been seen, the stored return value is returned instead of evaluating the original function.